# Why Use ML4Science's Tensors

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/holl-/ML4Science/blob/main/docs/Tensors.ipynb)
&nbsp; • &nbsp; [🌐 **ML4Science**](https://github.com/holl-/ML4Science)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/ML4Science/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/ML4Science/ml4s)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/holl-/ML4Science/blob/main/docs/Examples.ipynb) [**Examples**](https://holl-.github.io/ML4Science/Examples.html)

While you can call many ML4Science function directly with native tensors, such as Jax tensors or NumPy arrays, we recommend wrapping them in ML4Science tensors.
These provide several benefits over the native tensors and allow you to write easy-to-read, more concise, more explicit, less error-prone code.

For an introduction into tensors and dimensions, check out the [introduction notebook](Introduction.ht).

This notebook is work in progress. It will explain

* Comparisons to native libraries.
* show easy-to-read, more concise, more explicit, less error-prone code

In [1]:
%%capture
!pip install ml4s

## Named Dimensions

All tensor dimensions in ML4Science are required to have a name and type.
These properties are part of the [tensor shape](Shapes.html).
When creating a ML4Science tensor, you need to specify the names and types of all dimensions.

In [3]:
from ml4s import math
from ml4s.math import channel, batch, spatial, instance, dual  # dimension types

math.wrap([0, 1, 2], channel('integers'))

(0, 1, 2) along integersᶜ

In [9]:
data = math.random_uniform(batch(examples=2), spatial(x=4, y=3))
data

(examplesᵇ=2, xˢ=4, yˢ=3) 0.570 ± 0.216 (1e-01...1e+00)

## Printing Options

As you can see, ML4Science summarizes tensors by default and color-codes the result text.
The Python formatting options let you customize how a tensor is printed, with options being separated by colons.
Here are some examples:

In [10]:
print(f"{data:summary:color:shape:dtype:.5e}")

(examplesᵇ=2, xˢ=4, yˢ=3) float32 5.70418e-01 ± 2.15591e-01 (1.47583e-01...9.55640e-01)


In [17]:
print(f"{data:full:color:shape:dtype:.3f}")

examples=0  
 0.204, 0.148, 0.273, 0.764,
 0.612, 0.512, 0.838, 0.794,
 0.631, 0.954, 0.453, 0.585  along (xˢ=4, yˢ=3)
examples=1  
 0.824, 0.736, 0.382, 0.608,
 0.467, 0.956, 0.452, 0.453,
 0.422, 0.713, 0.467, 0.443  along (xˢ=4, yˢ=3)


In [21]:
print(f"{data:numpy:no-color:no-shape:no-dtype:.2f}")

[[[0.63 0.61 0.20]
  [0.95 0.51 0.15]
  [0.45 0.84 0.27]
  [0.58 0.79 0.76]]

 [[0.42 0.47 0.82]
  [0.71 0.96 0.74]
  [0.47 0.45 0.38]
  [0.44 0.45 0.61]]]


The order of the formatting arguments is not important.
Supported options are:

**Layout:**
The layout determines what is printed and where. The following options are available:

* `summary` Summarizes the values by mean, standard deviation, minimum and maximum value.
* `row` Prints the tensor as a single-line vector.
* `full` Prints all values in the tensors as a multi-line string.
* `numpy` Uses the formatting of NumPy

**Number format**:
You can additionally specify a format string for floating-point numbers like `.3f` or `.2e`.

**Color**:
Use the keywords `color` or `no-color`.
Currently `color` will use ANSI color codes which are supported by most terminals, IDEs as well as Jupyter notebooks.

**Additional tensor information**:
The keywords `shape`, `no-shape`, `dtype` and `no-dtype` can be used to show or hide additional properties of the tensor.


## Wrapping and Unwrapping

You can wrap existing tensors in ML4Science tensors using [`wrap()`](ml4s/math#ml4s.math.wrap) or [`tensor()`](ml4s/math#ml4s.math.tensor).
While `tensor()` will convert the data to the [default backend](Convert.html), `wrap()` will keep the data as-is.
In either case, you need to specify the dimension names and types when wrapping a native tensor.

In [24]:
math.use('torch')
math.tensor([0, 1, 2], batch('examples'))

(0, 1, 2) along examplesᵇ

To unwrap a tensor, you can use `tensor.native()` or `math.reshaped_native()` for more control over the result shape.
In both cases, the requested dimension order needs to be specified.

In [25]:
data.native('examples,x,y')

array([[[0.6309898 , 0.611722  , 0.2038249 ],
        [0.9543073 , 0.5123106 , 0.14758302],
        [0.45304176, 0.83762187, 0.27338478],
        [0.5846348 , 0.794242  , 0.7644842 ]],

       [[0.42182267, 0.46698892, 0.82417876],
        [0.7127511 , 0.9556401 , 0.7356606 ],
        [0.46661076, 0.45159382, 0.38228187],
        [0.4433869 , 0.45273894, 0.608222  ]]], dtype=float32)

Similarly, you can get the NumPy representation:

In [30]:
data.numpy('examples,x,y')

array([[[0.6309898 , 0.611722  , 0.2038249 ],
        [0.9543073 , 0.5123106 , 0.14758302],
        [0.45304176, 0.83762187, 0.27338478],
        [0.5846348 , 0.794242  , 0.7644842 ]],

       [[0.42182267, 0.46698892, 0.82417876],
        [0.7127511 , 0.9556401 , 0.7356606 ],
        [0.46661076, 0.45159382, 0.38228187],
        [0.4433869 , 0.45273894, 0.608222  ]]], dtype=float32)

## Further Reading

Check out the [examples](https://holl-.github.io/ML4Science/Examples.html) to see how using ML4Science's tensors is different from the other libraries.

Learn more about the [dimension types](Shapes.html).

ML4Science unifies [data types](Data_Types.html) as well and lets you set the floating point precision globally or by context.

While the dimensionality of neural networks must be specified during network creation, this is not the case for math functions.
These [automatically adapt to the number of spatial dimensions of the data that is passed in](N_Dimensional.html).

[🌐 **ML4Science**](https://github.com/holl-/ML4Science)
&nbsp; • &nbsp; [📖 **Documentation**](https://holl-.github.io/ML4Science/)
&nbsp; • &nbsp; [🔗 **API**](https://holl-.github.io/ML4Science/ml4s)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/holl-/ML4Science/blob/main/docs/Examples.ipynb) [**Examples**](https://holl-.github.io/ML4Science/Examples.html)